In [1]:
import sys
sys.path.append('../../../')

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from keras.layers import Dense, Dropout

import tensorflow as tf
import numpy as np
from tqdm import tqdm
from keras.layers import Dense, Dropout, GRU

import pandas as pd
import tensorflow as tf
import numpy as np

from machine_learning.models import DoubleConcatProduction
from data_processing.batch_generator import load_test_set
tf.random.set_seed(42)

define model:

In [3]:
model = DoubleConcatProduction()


In [4]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.TruePositives(),
            tf.keras.metrics.TrueNegatives(),
            tf.keras.metrics.FalsePositives(),
            tf.keras.metrics.FalseNegatives(),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC()])


### Dataset

In [5]:
path = '../../../'

In [6]:
sample_transaction = np.load(f'{path}data/test/all_transactions.npy')[0]
test_labels = np.load(f'{path}data/test/all_labels.npy').astype(float)
test_set = tf.data.Dataset.from_tensor_slices(
    (np.load(f'{path}data/test/all_transactions.npy'), 
     test_labels)
).batch(1)
# initialize weights
model(np.expand_dims(sample_transaction, axis=0))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.9999999]], dtype=float32)>

### Evaluate Model

In [7]:
train_df = pd.read_csv(f'{path}training_runs/train_double_production.csv')
train_df['f1_score'] = 2 * (train_df['precision'] * train_df['recall']) / (train_df['precision'] + train_df['recall'])
train_df

,epoch,loss,binary_accuracy,TP,TN,FP,FN,precision,recall,auc,f1_score
0,0,0.031267,0.994233,0.0,1013239.0,979.0,4898.0,0.000000,0.000000,0.648907,NaN
1,1,0.028978,0.995194,0.0,1014218.0,0.0,4898.0,0.000000,0.000000,0.671133,NaN
2,2,0.028706,0.995194,0.0,1014218.0,0.0,4898.0,0.000000,0.000000,0.683513,NaN
3,3,0.027816,0.995194,0.0,1014218.0,0.0,4898.0,0.000000,0.000000,0.708214,NaN
4,4,0.022309,0.995313,238.0,1014101.0,117.0,4660.0,0.670423,0.048591,0.809794,0.090615
5,5,0.019292,0.995459,477.0,1014011.0,207.0,4421.0,0.697368,0.097387,0.850065,0.170906
6,6,0.017771,0.996014,1534.0,1013520.0,698.0,3364.0,0.687276,0.313189,0.847243,0.430295
7,7,0.016832,0.996073,1688.0,1013426.0,792.0,3210.0,0.680645,0.344630,0.859633,0.457577
8,8,0.015825,0.996338,1902.0,1013482.0,736.0,2996.0,0.721001,0.388322,0.877993,0.504777
9,9,0.015572,0.996457,2217.0,1013288.0,930.0,2681.0,0.704480,0.452634,0.876082,0.551150


In [8]:
val_df = pd.read_csv(f'{path}training_runs/validation_{model.name}.csv')
val_df['f1_score'] = 2 * (val_df['precision'] * val_df['recall']) / (val_df['precision'] + val_df['recall'])
val_df

,epoch,loss,binary_accuracy,TP,TN,FP,FN,precision,recall,auc,f1_score
0,0,0.002485,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1,0.000681,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2,0.001310,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN
3,3,0.001153,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN
4,4,0.002603,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN
5,5,0.008247,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN
6,6,0.015251,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN
7,7,0.001620,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN
8,8,0.000318,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN
9,9,0.000386,1.0,0.0,91804.0,0.0,0.0,0.0,0.0,0.0,NaN


The epoch with the best performance on validation is **16**

In [9]:
model.load_weights(f'{path}machine_learning/saved_models/{model.name}_{17}.keras')

In [10]:
model.evaluate(test_set, return_dict=True)

555719/555719 [==============================] - 375s 673us/step - loss: 0.0184 - binary_accuracy: 0.9961 - true_positives: 4.0000 - true_negatives: 553574.0000 - false_positives: 0.0000e+00 - false_negatives: 2141.0000 - precision: 1.0000 - recall: 0.0019 - auc: 0.8355


{'loss': 0.018427060917019844,
 'binary_accuracy': 0.9961473345756531,
 'true_positives': 4.0,
 'true_negatives': 553574.0,
 'false_positives': 0.0,
 'false_negatives': 2141.0,
 'precision': 1.0,
 'recall': 0.0018648018594831228,
 'auc': 0.8354500532150269}